# **Time series weather product pridiction **
#### *Learning Objective: Three Tree algorithm application and ANN(MLP) 

## Step 1: Import neccsary pakageses

In [3]:
import pandas as pd
#Step 1: Read and explore data
file='C:/Users/Addisu/Desktop/kidi files/kidi/Evap/Data/Observed/Metehara/DEVAPll.csv'
raw_data = pd.read_csv(file, parse_dates = ['Date'],
                       index_col = 'Date')
dfml = raw_data.copy()
dfml

,PCP,Tmin,Tmax,Month,year,Tmean,Rhmax,Rhmin,Rhmean,Evapoavg,WS,SH
Date,,,,,,,,,,,,
1993-01-01,0.0,20.8,31.6,1,1993,26.20,75.0,40.0,57.5,3.471109,1.8,8.4
1993-01-02,0.0,17.8,32.0,1,1993,24.90,81.0,44.0,62.5,3.421044,1.6,8.3
1993-01-03,0.0,14.0,33.0,1,1993,23.50,89.0,41.0,65.0,4.001721,1.2,10.5
1993-01-04,0.0,12.6,32.0,1,1993,22.30,91.0,46.0,68.5,3.618233,2.1,9.2
1993-01-05,0.0,16.2,33.8,1,1993,25.00,82.0,43.0,62.5,3.844625,1.4,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27,0.0,10.5,31.0,12,2016,20.75,83.0,26.0,54.5,1.150054,0.8,0.1
2016-12-28,0.0,8.4,32.0,12,2016,20.20,86.0,27.0,56.5,3.723602,0.9,10.3
2016-12-29,0.0,9.0,32.0,12,2016,20.50,80.0,28.0,54.0,1.185701,1.0,0.2


## Gradient Booster Regressor

In [6]:
# GRADIENT BOOSTER
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn import metrics, svm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes  import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import cohen_kappa_score


# Read data from  csv files
X = dfml.drop(["Evapoavg"], axis=1)
y= dfml["Evapoavg"]
Input_data_features = X.values
Input_data_labels =y.values


# standardize input features X and output labels Y
#scaler_standardized_X = StandardScaler()
#Input_data_features = scaler_standardized_X.fit_transform(Input_data_features)

#scaler_standardized_Y = StandardScaler()
#Input_data_labels = scaler_standardized_Y.fit_transform(Input_data_labels)


# Split dataset into train, validation, an test
index_X_Train_End = int(0.9 * len(Input_data_features))
index_X_Validation_End = int(0.9 * len(Input_data_features))

X_train = Input_data_features[0: index_X_Train_End]
X_valid = Input_data_features[index_X_Train_End: index_X_Validation_End]
X_test = Input_data_features[index_X_Validation_End:]
Y_valid = Input_data_labels[index_X_Train_End: index_X_Validation_End]
Y_test = Input_data_labels[index_X_Validation_End:]
Y_train = Input_data_labels[0: index_X_Train_End]


# gradiant booster
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(gb.get_params())

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
               #'bootstrap': bootstrap}
pprint(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
gb = GradientBoostingRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
gb_random = RandomizedSearchCV(estimator = gb, 
                               param_distributions = random_grid, 
                               n_iter = 10, 
                               cv = 7, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs = -1)
# Fit the random search model
gb_random.fit(X_train, Y_train)


gb_random.best_params_

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = GradientBoostingRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, Y_train)
base_accuracy = evaluate(base_model,X_test, Y_test)
gbb = base_model.predict(X_test)

from sklearn import metrics

gbb = base_model.predict(X_test)
score = np.sqrt(metrics.mean_squared_error(Y_test,gbb))
print("Score (RMSE): {}".format(score))

from sklearn.metrics import mean_absolute_error

print('Test MAE  : %.3f'%mean_absolute_error(Y_test,gbb))
#print('Train MAE : %.3f'%mean_absolute_error(_train, lin_reg.predict(X_train)))

from sklearn.metrics import mean_squared_error, mean_squared_log_error
print('Test MSE  : %.3f'%mean_squared_error(Y_test,gbb))



best_random = gb_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, Y_test)
gbbe = best_random.predict(X_test)
from sklearn import metrics

rf = best_random.predict(X_test)
score = np.sqrt(metrics.mean_squared_error(Y_test,gbbe))
print("Score (RMSE): {}".format(score))

from sklearn.metrics import mean_absolute_error

print('Test MAE  : %.3f'%mean_absolute_error(Y_test,gbbe))
#print('Train MAE : %.3f'%mean_absolute_error(_train, lin_reg.predict(X_train)))

from sklearn.metrics import mean_squared_error, mean_squared_log_error
print('Test MSE  : %.3f'%mean_squared_error(Y_test,gbbe))

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))



def relative_root_mean_squared_error(actual, predictions,model_name):
    num = np.sum(np.square(actual - predictions))
    den = np.sum(np.square(predictions))
    squared_error = num/den
    rrmse = np.sqrt(squared_error)
    print(model_name + ':')
    print('Relative root mean square error-LSTM: {:.4f}'.format(rrmse))
    return rrmse
def nse(actual, predictions,model_name):
    nse=(1-(np.sum((actual-predictions)**2)/np.sum((actual-np.mean(actual))**2)))
    print(model_name + ':')
    print('Nash-Sutcliff-Efficiency-LSTM: {:.4f}'.format(nse))
    return nse
from scipy import stats
def calc_kge(actual, predictions,model_name):
    """Calculate the Kling-Gupta-Efficiency.
    
    Calculate the original KGE value following [1].

    Args:
        obs: Array of the observed values
        sim: Array of the simulated values

    Returns:
        The KGE value for the simulation, compared to the observation.

    Raises:
        ValueError: If the arrays are not of equal size or have non-numeric
            values.
        TypeError: If the arrays is not a supported datatype.
        RuntimeError: If the mean or the standard deviation of the observations
            equal 0.
    
    [1] Gupta, H. V., Kling, H., Yilmaz, K. K., & Martinez, G. F. (2009). 
    Decomposition of the mean squared error and NSE performance criteria: 
    Implications for improving hydrological modelling. Journal of Hydrology, 
    377(1-2), 80-91.
    
    """
    # Validation check on the input arrays
    #actual = validate_array_input(actual, np.float64, 'actual')
    #predictions = validate_array_input(predictions, np.float64, 'predictions')
    
    if len(actual) != len(predictions):
        raise ValueError("Arrays must have the same size.")
     
    mean_actual = np.mean(actual)
    if mean_actual == 0:
        msg = "KGE not definied if the mean of the observations equals 0."
        raise RuntimeError(msg)
    
    std_actual = np.std(actual)
    if std_actual == 0:
        msg = ["KGE not definied if the standard deviation of the ",
               "observations equals 0."]
        raise RuntimeError("".join(msg))
    actual = np.ndarray.flatten(actual)
    predictions = np.ndarray.flatten(predictions)
    r = stats.pearsonr(actual, predictions)[0]
    alpha = np.std(predictions) / std_actual
    beta = np.mean(predictions) / mean_actual
    
    kge_val = 1 - np.sqrt((r-1)**2 + (alpha-1)**2 + (beta-1)**2)
    print(model_name + ':')
    print('Kling-Gupta-Efficiency-LSTM: {:.4f}'.format(kge_val))
    return kge_val 
def pbias(actual, predictions,model_name):
    pbias=100 *(sum((actual - predictions)/sum(actual)))
    #print(model_name + ':')
    #print('Percent-of-Bias-bilstm: {:.4f}'.format(pbias))
    return pbias

nse(Y_test, gbbe,'LSTM')
calc_kge(Y_test, gbbe,'LSTM')
relative_root_mean_squared_error(Y_test, gbbe,'LSTM')
#evaluate(Y_test,rfr, 'LSTM')
PBIAS_BILSTM=pbias(Y_test, gbbe ,'LSTMLSTM')
PBIAS_BILSTM


nse(Y_test, gbb,'LSTM')
calc_kge(Y_test, gbb,'LSTM')
relative_root_mean_squared_error(Y_test, gbb,'LSTM')
#evaluate(Y_test,rfr, 'LSTM')
PBIAS_BILSTM=pbias(Y_test, gbb ,'LSTMLSTM')
PBIAS_BILSTM


Parameters currently in use:

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': 42,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}
{'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
Fitting 7 folds for each of 10 candidates, totalling 70 fits


C:\anaconda3\envs\jupyterenv\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
35 fits failed out of a total of 70.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "C:\anaconda3\envs\jupyterenv\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\anaconda3\envs\jupyterenv\lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\anaconda3\envs\jupyterenv\lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\anaconda3\envs\jupyteren

Model Performance
Average Error: 0.3112 degrees.
Accuracy = 90.06%.
Score (RMSE): 0.40657626316808543
Test MAE  : 0.311
Test MSE  : 0.165
Model Performance
Average Error: 0.0970 degrees.
Accuracy = 97.11%.
Score (RMSE): 0.16293416760063073
Test MAE  : 0.097
Test MSE  : 0.027
Improvement of 7.83%.
LSTM:
Nash-Sutcliff-Efficiency-LSTM: 0.9656
LSTM:
Kling-Gupta-Efficiency-LSTM: 0.9444
LSTM:
Relative root mean square error-LSTM: 0.0395
LSTM:
Nash-Sutcliff-Efficiency-LSTM: 0.7859
LSTM:
Kling-Gupta-Efficiency-LSTM: 0.5810
LSTM:
Relative root mean square error-LSTM: 0.0983


-0.9598815487477081

## Random forest regressor

In [7]:
#%% RANDOM FOREST 
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, Y_train)


rf_random.best_params_

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, Y_train)
base_accuracy = evaluate(base_model,X_test, Y_test)
rfr = base_model.predict(X_test)

#%%
from sklearn import metrics

rfr = base_model.predict(X_test)
score = np.sqrt(metrics.mean_squared_error(Y_test,rfr))
print("Score (RMSE): {}".format(score))

from sklearn.metrics import mean_absolute_error

print('Test MAE  : %.3f'%mean_absolute_error(Y_test,rfr))
#print('Train MAE : %.3f'%mean_absolute_error(_train, lin_reg.predict(X_train)))

from sklearn.metrics import mean_squared_error, mean_squared_log_error
print('Test MSE  : %.3f'%mean_squared_error(Y_test,rfr))
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))
def relative_root_mean_squared_error(actual, predictions,model_name):
    num = np.sum(np.square(actual - predictions))
    den = np.sum(np.square(predictions))
    squared_error = num/den
    rrmse = np.sqrt(squared_error)
    print(model_name + ':')
    print('Relative root mean square error-LSTM: {:.4f}'.format(rrmse))
    return rrmse
def nse(actual, predictions,model_name):
    nse=(1-(np.sum((actual-predictions)**2)/np.sum((actual-np.mean(actual))**2)))
    print(model_name + ':')
    print('Nash-Sutcliff-Efficiency-LSTM: {:.4f}'.format(nse))
    return nse
from scipy import stats
def calc_kge(actual, predictions,model_name):
    """Calculate the Kling-Gupta-Efficiency.
    
    Calculate the original KGE value following [1].

    Args:
        obs: Array of the observed values
        sim: Array of the simulated values

    Returns:
        The KGE value for the simulation, compared to the observation.

    Raises:
        ValueError: If the arrays are not of equal size or have non-numeric
            values.
        TypeError: If the arrays is not a supported datatype.
        RuntimeError: If the mean or the standard deviation of the observations
            equal 0.
    
    [1] Gupta, H. V., Kling, H., Yilmaz, K. K., & Martinez, G. F. (2009). 
    Decomposition of the mean squared error and NSE performance criteria: 
    Implications for improving hydrological modelling. Journal of Hydrology, 
    377(1-2), 80-91.
    
    """
    # Validation check on the input arrays
    #actual = validate_array_input(actual, np.float64, 'actual')
    #predictions = validate_array_input(predictions, np.float64, 'predictions')
    
    if len(actual) != len(predictions):
        raise ValueError("Arrays must have the same size.")
     
    mean_actual = np.mean(actual)
    if mean_actual == 0:
        msg = "KGE not definied if the mean of the observations equals 0."
        raise RuntimeError(msg)
    
    std_actual = np.std(actual)
    if std_actual == 0:
        msg = ["KGE not definied if the standard deviation of the ",
               "observations equals 0."]
        raise RuntimeError("".join(msg))
    actual = np.ndarray.flatten(actual)
    predictions = np.ndarray.flatten(predictions)
    r = stats.pearsonr(actual, predictions)[0]
    alpha = np.std(predictions) / std_actual
    beta = np.mean(predictions) / mean_actual
    
    kge_val = 1 - np.sqrt((r-1)**2 + (alpha-1)**2 + (beta-1)**2)
    print(model_name + ':')
    print('Kling-Gupta-Efficiency-LSTM: {:.4f}'.format(kge_val))
    return kge_val 
def pbias(actual, predictions,model_name):
    pbias=100 *(sum((actual - predictions)/sum(actual)))
    #print(model_name + ':')
    #print('Percent-of-Bias-bilstm: {:.4f}'.format(pbias))
    return pbias

nse(Y_test, rfr,'LSTM')
calc_kge(Y_test, rfr,'LSTM')
relative_root_mean_squared_error(Y_test, rfr,'LSTM')
#evaluate(Y_test,rfr, 'LSTM')
PBIAS_BILSTM=pbias(Y_test, rfr,'LSTMLSTM')
PBIAS_BILSTM

best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, Y_test)
rf = best_random.predict(X_test)
from sklearn import metrics

rf = best_random.predict(X_test)
score = np.sqrt(metrics.mean_squared_error(Y_test,rf))
print("Score (RMSE): {}".format(score))

print('Test MAE  : %.3f'%mean_absolute_error(Y_test,rf))
#print('Train MAE : %.3f'%mean_absolute_error(_train, lin_reg.predict(X_train)))

from sklearn.metrics import mean_squared_error, mean_squared_log_error
print('Test MSE  : %.3f'%mean_squared_error(Y_test,rf))

nse(Y_test, rf,'LSTM')
calc_kge(Y_test, rf,'LSTM')
relative_root_mean_squared_error(Y_test, rf,'LSTM')
#evaluate(Y_test,rfr, 'LSTM')
PBIAS_BILSTM=pbias(Y_test, rf,'LSTMLSTM')
PBIAS_BILSTM

#predictions=regressor.predict(send_to_model)

#print(predictions)
#output=pd.DataFrame({"EVP":rf})
#output.to_csv('C:/Users/Addisu/Desktop/kidi/Evap/Data/Data For Machine learning/Dubti/New Dubti/RF_IS2.csv',index=False)

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
Fitting 3 folds for each of 10 candidates, totalling 30 fits


C:\anaconda3\envs\jupyterenv\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
13 fits failed with the following error:
Traceback (most recent call last):
  File "C:\anaconda3\envs\jupyterenv\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\anaconda3\envs\jupyterenv\lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\anaconda3\envs\jupyterenv\lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\anaconda3\envs\jupyteren

Model Performance
Average Error: 0.0326 degrees.
Accuracy = 99.08%.
Score (RMSE): 0.04498803176108161
Test MAE  : 0.033
Test MSE  : 0.002
Improvement of -1.99%.
LSTM:
Nash-Sutcliff-Efficiency-LSTM: 0.9974
LSTM:
Kling-Gupta-Efficiency-LSTM: 0.9897
LSTM:
Relative root mean square error-LSTM: 0.0108
Model Performance
Average Error: 0.1052 degrees.
Accuracy = 96.40%.
Score (RMSE): 0.17056310390881613
Test MAE  : 0.105
Test MSE  : 0.029
LSTM:
Nash-Sutcliff-Efficiency-LSTM: 0.9623
LSTM:
Kling-Gupta-Efficiency-LSTM: 0.8912
LSTM:
Relative root mean square error-LSTM: 0.0413


0.3020464871311909

## XGBoost Regressor

In [8]:
# XGBOOST

from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
parameters = {'objective':['reg:squarederror'],
              'booster':['gbtree','gblinear'],
              'learning_rate': [0.1], 
              'max_depth': [7,10,15,20],
              'min_child_weight': [10,15,20,25],
              'colsample_bytree': [0.8, 0.9, 1],
              'n_estimators': [300,400,500,600],
              "reg_alpha"   : [0.5,0.2,1],
              "reg_lambda"  : [2,3,5],
              "gamma"       : [1,2,3]}

xgb_model = XGBRegressor(random_state=30)

grid_obj_xgb = RandomizedSearchCV(xgb_model,parameters, cv=5,n_iter=5,scoring='neg_mean_absolute_error',verbose=5,n_jobs=12)
grid_obj_xgb.fit(X_train, Y_train,verbose = 1)

#y_pred_train = grid_obj_xgb.predict(df_train)
y_pred_test = grid_obj_xgb.predict(X_test)

from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(xgb_model.get_params())

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = XGBRegressor(random_state=30)
base_model.fit(X_train, Y_train)
base_accuracy = evaluate(base_model,X_test, Y_test)
xgbb = base_model.predict(X_test)
score = np.sqrt(metrics.mean_squared_error(Y_test,xgbb))
print("Score (RMSE): {}".format(score))

from sklearn.metrics import mean_absolute_error

print('Test MAE  : %.3f'%mean_absolute_error(Y_test,xgbb))
#print('Train MAE : %.3f'%mean_absolute_error(_train, lin_reg.predict(X_train)))

from sklearn.metrics import mean_squared_error, mean_squared_log_error
print('Test MSE  : %.3f'%mean_squared_error(Y_test,xgbb))



y_pred_test = grid_obj_xgb.predict(X_test)
grid_obj_xgb.best_params_
grid_obj_xgb.best_score_

#best_random = kn_random.best_estimator_
random_accuracy = evaluate(grid_obj_xgb, X_test, Y_test)
y_pred_test = grid_obj_xgb.predict(X_test)
score = np.sqrt(metrics.mean_squared_error(Y_test,y_pred_test))
print("Score (RMSE): {}".format(score))

from sklearn.metrics import mean_absolute_error

print('Test MAE  : %.3f'%mean_absolute_error(Y_test,y_pred_test))
#print('Train MAE : %.3f'%mean_absolute_error(_train, lin_reg.predict(X_train)))

from sklearn.metrics import mean_squared_error, mean_squared_log_error
print('Test MSE  : %.3f'%mean_squared_error(Y_test,y_pred_test))


print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))
nse(Y_test,y_pred_test,'LSTM')
calc_kge(Y_test,y_pred_test,'LSTM')
relative_root_mean_squared_error(Y_test,y_pred_test,'LSTM')
#evaluate(Y_test,rfr, 'LSTM')
PBIAS_BILSTM=pbias(Y_test,y_pred_test,'LSTMLSTM')
PBIAS_BILSTM
nse(Y_test,xgbb,'LSTM')
calc_kge(Y_test,xgbb,'LSTM')
relative_root_mean_squared_error(Y_test,xgbb,'LSTM')
#evaluate(Y_test,rfr, 'LSTM')
PBIAS_BILSTM=pbias(Y_test,xgbb,'LSTMLSTM')
PBIAS_BILSTM
#output=pd.DataFrame({"EVP":y_pred_test})
#output.to_csv('C:/Users/Addisu/Desktop/kidi/Evap/Data/Data For Machine learning/Dubti/New Dubti/XGBoost_IS2.csv',index=False)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Parameters currently in use:

{'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'objective': 'reg:squarederror',
 'predictor': None,
 'random_state': 30,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,

0.07894068593436875

## MLP Regressor

In [9]:
# MLP
from sklearn.neural_network import MLPRegressor
mlpr = MLPRegressor(max_iter=7000)
from sklearn.model_selection import RandomizedSearchCV
parameter_space = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

gridCV = RandomizedSearchCV( mlpr,parameter_space,n_jobs=-1, cv=5 )
gridCV.fit(X_train, Y_train)
mlpr.fit(X_train, Y_train)
random_accuracy = evaluate(mlpr, X_test, Y_test)
mlpp= mlpr.predict(X_test)
from sklearn import metrics

rf = best_random.predict(X_test)
score = np.sqrt(metrics.mean_squared_error(Y_test,mlpp))
print("Score (RMSE): {}".format(score))

from sklearn.metrics import mean_absolute_error

print('Test MAE  : %.3f'%mean_absolute_error(Y_test,mlpp))
#print('Train MAE : %.3f'%mean_absolute_error(_train, lin_reg.predict(X_train)))

from sklearn.metrics import mean_squared_error, mean_squared_log_error
print('Test MSE  : %.3f'%mean_squared_error(Y_test,mlpp))

#print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))
score = np.sqrt(metrics.mean_squared_error(Y_test,mlpp))
print("Score (RMSE): {}".format(score))

from sklearn.metrics import mean_absolute_error

print('Test MAE  : %.3f'%mean_absolute_error(Y_test,mlpp))
#print('Train MAE : %.3f'%mean_absolute_error(_train, lin_reg.predict(X_train)))

from sklearn.metrics import mean_squared_error, mean_squared_log_error
print('Test MSE  : %.3f'%mean_squared_error(Y_test,mlpp))

best_random = gridCV.best_estimator_
random_accuracy = evaluate(best_random, X_test, Y_test)
mlp= best_random.predict(X_test)
from sklearn import metrics

rf = best_random.predict(X_test)
score = np.sqrt(metrics.mean_squared_error(Y_test,mlp))
print("Score (RMSE): {}".format(score))

from sklearn.metrics import mean_absolute_error

print('Test MAE  : %.3f'%mean_absolute_error(Y_test,mlp))
#print('Train MAE : %.3f'%mean_absolute_error(_train, lin_reg.predict(X_train)))

from sklearn.metrics import mean_squared_error, mean_squared_log_error
print('Test MSE  : %.3f'%mean_squared_error(Y_test,mlp))

#print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

def relative_root_mean_squared_error(actual, predictions,model_name):
    num = np.sum(np.square(actual - predictions))
    den = np.sum(np.square(predictions))
    squared_error = num/den
    rrmse = np.sqrt(squared_error)
    print(model_name + ':')
    print('Relative root mean square error-LSTM: {:.4f}'.format(rrmse))
    return rrmse
def nse(actual, predictions,model_name):
    nse=(1-(np.sum((actual-predictions)**2)/np.sum((actual-np.mean(actual))**2)))
    print(model_name + ':')
    print('Nash-Sutcliff-Efficiency-LSTM: {:.4f}'.format(nse))
    return nse
from scipy import stats
def calc_kge(actual, predictions,model_name):
    """Calculate the Kling-Gupta-Efficiency.
    
    Calculate the original KGE value following [1].

    Args:
        obs: Array of the observed values
        sim: Array of the simulated values

    Returns:
        The KGE value for the simulation, compared to the observation.

    Raises:
        ValueError: If the arrays are not of equal size or have non-numeric
            values.
        TypeError: If the arrays is not a supported datatype.
        RuntimeError: If the mean or the standard deviation of the observations
            equal 0.
    
    [1] Gupta, H. V., Kling, H., Yilmaz, K. K., & Martinez, G. F. (2009). 
    Decomposition of the mean squared error and NSE performance criteria: 
    Implications for improving hydrological modelling. Journal of Hydrology, 
    377(1-2), 80-91.
    
    """
    # Validation check on the input arrays
    #actual = validate_array_input(actual, np.float64, 'actual')
    #predictions = validate_array_input(predictions, np.float64, 'predictions')
    
    if len(actual) != len(predictions):
        raise ValueError("Arrays must have the same size.")
     
    mean_actual = np.mean(actual)
    if mean_actual == 0:
        msg = "KGE not definied if the mean of the observations equals 0."
        raise RuntimeError(msg)
    
    std_actual = np.std(actual)
    if std_actual == 0:
        msg = ["KGE not definied if the standard deviation of the ",
               "observations equals 0."]
        raise RuntimeError("".join(msg))
    actual = np.ndarray.flatten(actual)
    predictions = np.ndarray.flatten(predictions)
    r = stats.pearsonr(actual, predictions)[0]
    alpha = np.std(predictions) / std_actual
    beta = np.mean(predictions) / mean_actual
    
    kge_val = 1 - np.sqrt((r-1)**2 + (alpha-1)**2 + (beta-1)**2)
    print(model_name + ':')
    print('Kling-Gupta-Efficiency-LSTM: {:.4f}'.format(kge_val))
    return kge_val 
def pbias(actual, predictions,model_name):
    pbias=100 *(sum((actual - predictions)/sum(actual)))
    #print(model_name + ':')
    #print('Percent-of-Bias-bilstm: {:.4f}'.format(pbias))
    return pbias

nse(Y_test, mlpp ,'LSTM')
calc_kge(Y_test,mlpp ,'LSTM')
relative_root_mean_squared_error(Y_test,mlpp ,'LSTM')
#evaluate(Y_test,rfr, 'LSTM')
PBIAS_BILSTM=pbias(Y_test, mlpp  ,'LSTMLSTM')

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

PBIAS_BILSTM=pbias(Y_test,mlpp,'LSTMLSTM')
PBIAS_BILSTM

nse(Y_test, mlp ,'LSTM')
calc_kge(Y_test,mlp ,'LSTM')
relative_root_mean_squared_error(Y_test,mlp ,'LSTM')
#evaluate(Y_test,rfr, 'LSTM')
PBIAS_BILSTM=pbias(Y_test, mlp  ,'LSTMLSTM')
#output=pd.DataFrame({"EVP":mlp})
#output.to_csv('C:/Users/Addisu/Desktop/kidi/Evap/Data/Data For Machine learning/Dubti/New Dubti/MLP_IS2.csv',index=False)

C:\anaconda3\envs\jupyterenv\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
3 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\anaconda3\envs\jupyterenv\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\anaconda3\envs\jupyterenv\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\anaconda3\envs\jupyterenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 753, in fit
    return self._fit(X, y, increment

Model Performance
Average Error: 0.2063 degrees.
Accuracy = 94.26%.
Score (RMSE): 0.2640159060574793
Test MAE  : 0.206
Test MSE  : 0.070
Score (RMSE): 0.2640159060574793
Test MAE  : 0.206
Test MSE  : 0.070
Model Performance
Average Error: 0.2310 degrees.
Accuracy = 93.44%.
Score (RMSE): 0.28525420169554694
Test MAE  : 0.231
Test MSE  : 0.081
LSTM:
Nash-Sutcliff-Efficiency-LSTM: 0.9097
LSTM:
Kling-Gupta-Efficiency-LSTM: 0.9193
LSTM:
Relative root mean square error-LSTM: 0.0623
LSTM:
Nash-Sutcliff-Efficiency-LSTM: 0.8946
LSTM:
Kling-Gupta-Efficiency-LSTM: 0.8644
LSTM:
Relative root mean square error-LSTM: 0.0707
